In [ ]:
import tensorflow_datasets as tfds

dataset_name = "cars196"
(ds_train, ds_test), ds_info = tfds.load(dataset_name, split=["train", "test"], with_info=True, as_supervised=True)
NUM_CLASSES = ds_info.features["label"].num_classes

In [ ]:
!wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet\
       /noisystudent/noisy_student_efficientnet-b0.tar.gz
!tar -xf noisy_student_efficientnet-b0.tar.gz


!python efficientnet_weight_update_util.py --model b1 --notop --ckpt \
        efficientnet-b1/model.ckpt --o efficientnetb1_notop.h5



In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import EfficientNetB0
from keras import layers, models, Model, optimizers, losses, metrics, regularizers, callbacks, initializers

IMG_SIZE: int = 224
BATCH_SIZE: int = 64
EPOCHES: int = 14  # @param {type: "slider", min:1, max:100}
top_dropout_rate: np.float = 0.2


metrics = [
    metrics.FalseNegatives(name="fn"),
    metrics.FalsePositives(name="fp"),
    metrics.TrueNegatives(name="tn"),
    metrics.TruePositives(name="tp"),
    metrics.Precision(name="precision"),
    metrics.Recall(name="recall"),
    metrics.Accuracy(name='acc'),
    metrics.AUC(name='auc'),
    metrics.AUC(name='prc', curve='PR')
]


def input_preprocess(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label


def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()


def plot_metrics(history):
  metrics = ['loss', 'prc', 'precision', 'recall']
  colors = ['b', 'g', 'r', 'y']
  plt.figure(figsize=(8, 8))
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch, history.history[metric], color='b', label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color='r', linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0.8,1])
    else:
      plt.ylim([0,1])
    plt.grid()
    plt.legend()


img_augmentation = models.Sequential(
    [
        layers.experimental.preprocessing.RandomRotation(factor=0.15),
        layers.experimental.preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.experimental.preprocessing.RandomFlip(),
        layers.experimental.preprocessing.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)


def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = img_augmentation(inputs)
    model = EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet") # weights="efficientnetb1_notop.h5"
    
    model.trainable = False

    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

    model = Model(inputs, outputs, name="EfficientNet")
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-2),
        loss="categorical_crossentropy",
        metrics=metrics
    )
    return model


def unfreeze_model(model):
    for layer in model.layers:
      if not isinstance(layer, layers.BatchNormalization):
          layer.trainable = True

    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-2),
        loss="categorical_crossentropy",
        metrics=metrics)
    return model


def plot_loss(history: callbacks.History) -> plt.plot:
  plt.semilogy(history.epoch, history.history['loss'],
               color='b', label='Train')
  plt.semilogy(history.epoch, history.history['val_loss'],
               color='r', label='Val',
               linestyle="--")
  plt.xlabel('Epoch')
  plt.legend()
  plt.grid()
  plt.title("Trai and validation loss")
  plt.ylabel('Loss')


def plot_prc(name, labels, predictions, **kwargs):
    precision, recall, _ = precision_recall_curve(labels, predictions)

    plt.plot(precision, recall, label=name, linewidth=2, **kwargs)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')

In [4]:
size = (IMG_SIZE, IMG_SIZE)
ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))

ds_train = ds_train.map(input_preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.batch(batch_size=BATCH_SIZE, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(input_preprocess)
ds_test = ds_test.batch(batch_size=BATCH_SIZE, drop_remainder=True)

In [ ]:
# with strategy.scope():
model = build_model(num_classes=NUM_CLASSES)
# model = unfreeze_model(model) For Unfreezing layers
model.summary()

model_history = model.fit(ds_train,
                          epochs=EPOCHES,
                          validation_data=ds_test,
                          verbose=2,
                          shuffle=True)

In [ ]:
# Plot Metrics

plot_loss(model_history)
plot_metrics(model_history)
plot_history(model_history)